In [2]:
# Install system-wide poppler
!apt-get -y install poppler-utils

# Install Python Libraries
!pip install pdf2image opencv-python numpy Pillow reportlab


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (1,779 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import cv2
from pdf2image import convert_from_path
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import os
from google.colab import files
import zipfile

# 色盲模拟矩阵
COLORBLIND_TRANSFORMS = {
    "protanopia": np.array([
        [0.567, 0.433, 0.000],
        [0.558, 0.442, 0.000],
        [0.000, 0.242, 0.758]
    ]),
    "deuteranopia": np.array([
        [0.625, 0.375, 0.000],
        [0.700, 0.300, 0.000],
        [0.000, 0.300, 0.700]
    ]),
    "tritanopia": np.array([
        [0.950, 0.050, 0.000],
        [0.000, 0.433, 0.567],
        [0.000, 0.475, 0.525]
    ]),
    "achromatopsia": "grayscale"
}

def simulate_colorblind(image: np.ndarray, cb_type: str) -> np.ndarray:
    if cb_type == "achromatopsia":
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    else:
        transform = COLORBLIND_TRANSFORMS[cb_type]
        img = image.astype(float) / 255.0
        simulated = img @ transform.T
        return (np.clip(simulated, 0, 1) * 255).astype(np.uint8)

def generate_simulated_pdf(pages, cb_type: str):
    output_files = []

    cb_types = list(COLORBLIND_TRANSFORMS.keys()) if cb_type == "all" else [cb_type]

    for cb in cb_types:
        image_paths = []
        for i, page in enumerate(pages):
            img = np.array(page.convert('RGB'))[:, :, ::-1]
            simulated = simulate_colorblind(img, cb)
            img_path = f"temp_{cb}_page_{i}.png"
            cv2.imwrite(img_path, simulated)
            image_paths.append(img_path)

        output_pdf = f"simulated_{cb}.pdf"
        c = canvas.Canvas(output_pdf, pagesize=letter)
        for path in image_paths:
            c.drawImage(path, 0, 0, width=612, height=792)
            c.showPage()
            os.remove(path)
        c.save()
        output_files.append(output_pdf)

    if cb_type == "all":
        zip_name = "colorblind_simulations.zip"
        with zipfile.ZipFile(zip_name, 'w') as zipf:
            for f in output_files:
                zipf.write(f)
        return zip_name
    else:
        return output_files[0]

# 👉 设置色盲类型：'protanopia' / 'deuteranopia' / 'tritanopia' / 'achromatopsia' / 'all'
cb_type = "all"

# 上传 PDF
uploaded = files.upload()
pdf_file = list(uploaded.keys())[0]

# 转换 PDF 页面
pages = convert_from_path(pdf_file, dpi=200)

# 模拟并输出文件
output_file = generate_simulated_pdf(pages, cb_type)
files.download(output_file)


Saving AAAI_2026_Latex2Layout_PDF_Parsing.pdf to AAAI_2026_Latex2Layout_PDF_Parsing.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>